# Annual Emissions Analysis

This notebook demonstrates how to use the emissions calculator to analyze operational emissions data.

In [ ]:
import sys
sys.path.insert(0, '..')

from src.data_loader import load_data_from_directory
from src.emissions_calculator import calculate_annual_emissions
import pandas as pd

## 1. Load Data

In [ ]:
# Load all CSV files from the data directory
dataframes = load_data_from_directory('../data/raw')

print(f"\nLoaded {len(dataframes)} dataframes:")
for name in sorted(dataframes.keys()):
    print(f"  - {name}: {dataframes[name].shape}")

## 2. Explore Operation Data

In [ ]:
# View operation data
operation_data = dataframes['operation_data_test_2']
print("Operation Data Shape:", operation_data.shape)
print("\nColumns:")
print(operation_data.columns.tolist())
print("\nFirst few rows:")
operation_data.head()

In [ ]:
# Check entity types
print("Entity types:")
print(operation_data['entity'].value_counts())

## 3. Calculate Annual Emissions

In [ ]:
# Calculate emissions
annual_emissions = calculate_annual_emissions(
    dataframes['operation_data_test_2'],
    dataframes['fuel_data'],
    dataframes['refrigerant_gwp'],
    dataframes['refrigerator_data'],
    dataframes['vehicle_interventions'],
    dataframes['vehicle_interventions_tru'],
    dataframes['farm_emissions']
)

print("\nEmissions calculated!")
annual_emissions.head()

## 4. Analyze Results

In [ ]:
# Summary statistics
print("Summary Statistics:")
print(annual_emissions.describe())

In [ ]:
# Total emissions by category
emission_categories = [
    'fuel_emissions', 'vehicle_emissions', 'refrigerant_emissions',
    'livestock_emissions', 'fertilizer_emissions', 'waste_emissions',
    'tru_emissions'
]

category_totals = annual_emissions[emission_categories].sum()
print("\nTotal Emissions by Category (kg CO2e):")
for category, total in category_totals.items():
    print(f"  {category}: {total:,.2f}")
print(f"\nTotal: {annual_emissions['total_emissions'].sum():,.2f}")

In [ ]:
# Visualize emissions breakdown
import matplotlib.pyplot as plt

# Pie chart of emission sources
non_zero_categories = category_totals[category_totals > 0]

plt.figure(figsize=(10, 6))
plt.pie(non_zero_categories, labels=non_zero_categories.index, autopct='%1.1f%%')
plt.title('Emissions by Source Category')
plt.show()

## 5. Save Results

In [ ]:
# Save to CSV
output_path = '../data/outputs/annual_emissions.csv'
annual_emissions.to_csv(output_path, index=False)
print(f"Results saved to: {output_path}")